In [11]:
import time
import pandas as pd
import csv
import sys
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import tree, metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str,
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])

In [12]:
df.head(10)

,instance,text,id,sentiment,is_sarcastic
0,1,Liberals might not be talking about #Parakeeli...,10000,negative,false
1,2,".@TurnbullMalcolm to CFA volunteers: ""Your sel...",10006,neutral,false
2,3,Peter Dutton's been “An Outstanding Immigrati...,10008,neutral,false
3,4,"If we are to respect women, we must respect S9...",10008,negative,false
4,5,#auspol @TurnbullMalcolm @Barnaby_Joyce We nee...,10000,negative,false
5,6,@ABC24 @PM. Chaotic hung Govt? Most chaos was ...,10006,negative,false
6,7,Fascist class warfare killings has only just b...,10006,negative,false
7,8,"Why the #LNP will not stand up to the banks, d...",10002,negative,false
8,9,?Turnbull couldn't find a real tradie- now con...,10001,negative,false
9,10,"While we're asking about #BorderFarce fiasco, ...",10001,negative,false


In [15]:
train = df[:1500]

In [16]:
test = df[1500:]

In [23]:
train = train[train.sentiment != 'neutral']
train

,instance,text,id,sentiment,is_sarcastic
0,1,Liberals might not be talking about #Parakeeli...,10000,negative,false
3,4,"If we are to respect women, we must respect S9...",10008,negative,false
4,5,#auspol @TurnbullMalcolm @Barnaby_Joyce We nee...,10000,negative,false
5,6,@ABC24 @PM. Chaotic hung Govt? Most chaos was ...,10006,negative,false
6,7,Fascist class warfare killings has only just b...,10006,negative,false
7,8,"Why the #LNP will not stand up to the banks, d...",10002,negative,false
8,9,?Turnbull couldn't find a real tradie- now con...,10001,negative,false
9,10,"While we're asking about #BorderFarce fiasco, ...",10001,negative,false
10,11,Suddenly the media blackout on arriving refuge...,10008,negative,false
11,12,There has never been a more exciting time for ...,10000,negative,true


In [21]:
test = test[test.sentiment != 'neutral']
test.shape

(375, 5)

In [4]:
df = df[df.sentiment != 'neutral']

In [5]:
df.head(10)

,instance,text,id,sentiment,is_sarcastic
0,1,Liberals might not be talking about #Parakeeli...,10000,negative,false
3,4,"If we are to respect women, we must respect S9...",10008,negative,false
4,5,#auspol @TurnbullMalcolm @Barnaby_Joyce We nee...,10000,negative,false
5,6,@ABC24 @PM. Chaotic hung Govt? Most chaos was ...,10006,negative,false
6,7,Fascist class warfare killings has only just b...,10006,negative,false
7,8,"Why the #LNP will not stand up to the banks, d...",10002,negative,false
8,9,?Turnbull couldn't find a real tradie- now con...,10001,negative,false
9,10,"While we're asking about #BorderFarce fiasco, ...",10001,negative,false
10,11,Suddenly the media blackout on arriving refuge...,10008,negative,false
11,12,There has never been a more exciting time for ...,10000,negative,true


In [10]:
pd.set_option('display.max_rows', 500)

,instance,text,id,sentiment,is_sarcastic
0,1,Liberals might not be talking about #Parakeeli...,10000,negative,false
3,4,"If we are to respect women, we must respect S9...",10008,negative,false
4,5,#auspol @TurnbullMalcolm @Barnaby_Joyce We nee...,10000,negative,false
5,6,@ABC24 @PM. Chaotic hung Govt? Most chaos was ...,10006,negative,false
6,7,Fascist class warfare killings has only just b...,10006,negative,false
7,8,"Why the #LNP will not stand up to the banks, d...",10002,negative,false
8,9,?Turnbull couldn't find a real tradie- now con...,10001,negative,false
9,10,"While we're asking about #BorderFarce fiasco, ...",10001,negative,false
10,11,Suddenly the media blackout on arriving refuge...,10008,negative,false
11,12,There has never been a more exciting time for ...,10000,negative,true


In [3]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [4]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = r'''!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = remove_punctuation(sample)
    return sample


def myTokenizer(sample):
    """Customized tokenizer"""
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2]
    return new_words


In [ ]:
try:
    size = int(sys.argv[1])
except IndexError:
    size = None
size = 200
count = CountVectorizer(preprocessor=myPreprocessor,
                        lowercase=False, tokenizer=myTokenizer, max_features=size)